# Advanced explanation for RCM
Contrary to Sentinel-1, Radarsat Constellation Mission doesn't have the notion of multi dataset

In [ ]:
import xsar
import geoviews as gv
import holoviews as hv
import geoviews.feature as gf
hv.extension('bokeh')
path = xsar.get_test_file('RCM1_OK1050603_PK1050605_1_SC50MB_20200214_115905_HH_HV_Z010')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Access metadata from a product
Raw information is stocked in different files such as tiff ones (for digital numbers).
A file named product.xml is constitued of the main information (geolocation grid, orbit attitude, noise look up tables...).
Calibration look up tables are located in xml files.
All the useful data is grouped in a datatree thanks to dependencie [xarray-safe-rcm](https://github.com/umr-lops/xarray-safe-rcm).
This datatree is than used as an attribute of [RcmMeta](../basic_api.rst#xsar.RcmMeta).

In [ ]:
#Instanciate a RadarSat2Meta object
meta = xsar.RcmMeta(name=path)

In [ ]:
#Access the datatree extracted from the reader
meta.dt

### Examples of alias to datasets (from the datatree above)

In [ ]:
#geolocation grid (low resolution)
meta.geoloc

In the metadata class : noise lut, calibration lut and incidence are processed because samples are expressed thanks to a firstPixelValue, a step and a nbOfValues.
It is made internally to the reader's datatree thanks to the method [assign_index](../basic_api.rst#xsar.RcmMeta.assign_index)
See Documentation : RCM Image Product Format Definition (7.5.1)

In [ ]:
#Calibration look up tables in range
meta.lut

In [ ]:
#noise look up tables in range
meta.noise_lut

In [ ]:
#incidence angles
meta.incidence

## Open a dataset

In [ ]:
# Define the resolution to load the dataset at a lower resolution (if not specified or None, the dataset is loaded at high resolution)
resolution = '1000m'

# Instanciate a RadarSatDataset object
rcmds = xsar.RcmDataset(dataset_id=path, resolution=resolution)

### Get the Dataset object

In [ ]:
rcmds

### Access the metadata object from the Dataset object

In [ ]:
rcmds.sar_meta

### Access the dataset
In this dataset, we can find variables like latitude, longitude, look up tables (before and after denoising), incidence...

In [ ]:
rcmds.dataset

Variables `lines_flipped`and `samples_flipped` are added to the dataset to know if these have been flipped (in order to follow xsar convention).

See RCM Image Product Format Definition (4.2.1) for more explication about the different cases when lines or samples are flipped

### Alternatives solutions to open dataset and datatree

In [ ]:
# Open dataset
xsar.open_dataset(path, resolution=resolution)

In [ ]:
# Open datatree
xsar.open_datatree(path, resolution=resolution)

## How to apply calibration?

All the operation below are already performed by default for GRD products. So, what is following is a simple explanation about how is made calibration.

### Load digital numbers 

[load_digital_number](../basic_api.rst#xsar.RcmDataset.load_digital_number) is a function that allows to load digital numbers from tiff files at chosen resolution and return it as a `DataArray`. 
Resampling is made thanks to `rasterio.open(tiff_file).read`. 
For dual pol products, there is a tiff file for each pol. So that digital numbers are computed for both pol. Posting of lines and samples is computed thanks to `Affine.translation` and `Affine.scale`.

In [ ]:
import rasterio

#Define resampling method (here it is the root mean square from rasterio)
resampling = rasterio.enums.Resampling.rms

#Define the chunks size for line and samples
chunks = {'line': 5000, 'sample': 5000}

In [ ]:
dn_low_res = rcmds.load_digital_number(resolution=resolution, resampling=resampling, chunks=chunks)
dn_low_res

### Get the raw normalized radar cross section

First the calibration look up tables are loaded at the good resolution in `xsar.RcmDataset._luts` thanks to the method [lazy_load_luts](../basic_api.rst#xsar.RcmDataset.lazy_load_luts). 
The resampling is made thanks to an interpolation with the method ̀`xsar.RcmDataset._interpolate_var`.

This last method uses `RectBivariateSpline`for the interpolation, but it is necessary that data is expressed as a 2D vector. Here, calibration/noise look up tables and incidence are expressed as 1D vector (range sample). Consequently, we need to convert these in 2D (adding an azimuth dimension dependency) before applying the interpolation. Conversion is made thanks to `numpy.tile`, using the low resolution lines expressed in the geolocation grid part of the reader; reducing the calculation. A template of a `DataArray` that uses the posting of digital numbers (with applied resolution) is given on this interpolation function so the result is now at the right resolution.

`_apply_calibration_lut` is a method that applies a calibration look up table to digital numbers to compute gamma0, beta0 and sigma0 (depending on the variable name passed in argument) and return the result as a `DataArray`. It applies the following formula :

$$ \frac{(digitalNumbers^2)+offset}{Gain} $$



Reference : `RCM Image Product Format Definition` (7.5.1)

Different resampling method were tried such as `scipy.ndimage.gaussian_filter1d` that had the convenience to accept 1d vectors. Data was computed with this function and the chosen posting was this of digital numbers. But in order to be homogenous with other operations made in `xsar`, we chose to keep the solution with `RectBivariateSpline`.

In [ ]:
sigma0_raw = rcmds._apply_calibration_lut('sigma0').sigma0_raw
sigma0_raw

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(13, 6)) 

plt.subplot(1, 2, 1)
sigma0_raw_cross = sigma0_raw.sel(pol='HV')
plt.pcolor(sigma0_raw_cross, vmax=0.02, cmap='Greys_r')
plt.title('sigma0_raw HV')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

plt.subplot(1, 2, 2)
sigma0_raw_co = sigma0_raw.sel(pol='HH')
plt.pcolor(sigma0_raw_co, vmax=0.7, cmap='Greys_r')
plt.title('sigma0_raw HH')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

## How to apply denoising ?

All the operation below are already performed by default for GRD products. So, what is following is a simple explanation about how is made denoising.

### How to get the Noise Equivalent Sigma Zero ?

The noise look up tables are loaded at the good resolution in `xsar.RcmDataset._noise_luts` thanks to the method [lazy_load_noise_luts](../basic_api.rst#xsar.RcmDataset.lazy_load_noise_luts). 
The resampling is made thanks to an interpolation with the method ̀`xsar.RcmDataset._interpolate_var`.

Notes:
- noise luts are already calibrated so we don't have to apply a calibration on these
- `interpolate_var` method explained above converts the NoiseLevelValues in linear beacause these are expressed in `dB` in the reader. The formula used is : 
    <font size="5">
    $$ NoiseLevelValues_{linear} = 10^\frac{NoiseLevelValues_{dB}}{10} $$
    </font>

In [ ]:
nesz_low_res = rcmds.lazy_load_noise_luts().sigma0
nesz_low_res

In [ ]:
plt.pcolor(nesz_low_res.sel(pol='HH'), cmap='Greys_r')
plt.title('nesz (pol = HH)')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

### How to get the noise substracted Sigma0

Right now we only have to substract the noise_lut to the raw normalized radar cross section. It is made with the function `_add_denoised`, that add the variables to the [RcmDataset.dataset](../basic_api.rst#xsar.RcmDataset.dataset)

In [ ]:
sigma0 = sigma0_raw - nesz_low_res
sigma0

### Comparison between noised sigma0 and noised substracted sigma0

In [ ]:
plt.figure(figsize=(26, 12))

sigma0_cross = sigma0.sel(pol='HV')
sigma0_co = sigma0.sel(pol='HH')

plt.subplot(2,2,1)
plt.pcolor(sigma0_raw_cross, vmax=0.02, cmap='Greys_r')
plt.title('Sigma0 HV with noise')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

plt.subplot(2,2,3)
plt.pcolor(sigma0_cross, vmax=0.02, cmap='Greys_r')
plt.title('Sigma0 HV without noise')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

plt.subplot(2,2,2)
plt.pcolor(sigma0_raw_co, vmax=0.7, cmap='Greys_r')
plt.title('Sigma0 HH with noise')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

plt.subplot(2,2,4)
plt.pcolor(sigma0_co, vmax=0.7, cmap='Greys_r')
plt.title('Sigma0 HH without noise')
plt.xlabel('samples')
plt.ylabel('lines')
plt.colorbar()

## How to get the incidence ?

Such as noise/calibration look up tables, incidence depends on samples, which are expressed in the reader thanks to step, firstPixelValue and nbOfValues. Samples are computed in the meta class directly on the datatree thanks to the method [assign_index](../basic_api.rst#xsar.RcmMeta.assign_index).

`_load_incidence_from_lut` is a function that applies an interpolation with `_interpolate_var` and then return the incidence at the good resolution

In [ ]:
incidence = rcmds._load_incidence_from_lut()
incidence

## How to get the elevation ?

To get the incidence, we apply a formula :
    
$$ \theta = \arcsin{[\sin{(Incidence)} . \frac{r}{r + h}]} $$

$$ ( r \text{ is the earth radius} , h \text{ is the orbit altitude} ) $$

2 variables give orbit altitude so we considered the`SatelliteHeight` (and not the`altitude`).


`RcmDataset._load_elevation_from_lut` permit to calculate the elevation (in degrees).

In [ ]:
elevation = rcmds._load_elevation_from_lut()
elevation